In [1]:
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

c:\users\son\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import random
import os

In [3]:
import keras
from keras.preprocessing.text import Tokenizer
import string
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, LSTM,Input,Conv1D,GlobalMaxPooling1D,BatchNormalization,MaxPooling1D,Flatten
from keras.layers.wrappers import Bidirectional 
from keras.callbacks import ModelCheckpoint,EarlyStopping
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Model
from keras.callbacks import EarlyStopping
import re
%matplotlib inline
import numpy as np
import pandas as pd
from keras.utils import to_categorical
import h5py

In [4]:
test_files = os.listdir('.\processed_test_parts')

In [5]:
test_files[0]

'part1.pkl'

In [6]:
n_unique_words = 10000 
max_text_length = 4 
pad_type = trunc_type = 'pre'
def pad_seq(sequence):
    return pad_sequences(sequence,maxlen=max_text_length,dtype='int32',padding='pre',truncating='pre',value=0.)

from keras.models import load_model

output_dir = 'model_output/parallel/t50/'
model=load_model(output_dir+"best_model.hdf5")

In [13]:
import time
results_dir = "results\\"
t0 = time.time()
for k in range(len(test_files)):
    t1 = time.time()
    print("loading "+str(k+1)+"th file...")
    fit_data = pd.read_pickle(  '.\\processed_test_parts\\'+'part'+str(k+1)+'.pkl' )
    print("file loaded. mapping categoricals")
    fit_data['cat'] = fit_data['app'].map(str)+'a '+fit_data['channel'].map(str)+'c '+ fit_data['device'].map(str)+'d '+\
    fit_data['os'].map(str)+['os ']
    print("mapping done. Tokenizing now.")
    fit_data['token_vec'] = tokenizer.texts_to_sequences(fit_data.cat.astype(str))
    test_cat = pad_seq(fit_data.token_vec)
    test_feat =fit_data[['ip','app_count','device_count','os_count','channel_count','click_count','apps',
                 'devices','oss','channels','adoc','app_count_min','device_count_min','os_count_min',
                  'channel_count_min','click_count_min','adoc_min','click_count_cumsum','click_count_cumsum_min'
                 ]].values
    
    print("predicting")
    y = model.predict([test_cat,test_feat])
    print("merging click id")
    y = pd.DataFrame(y)
    y.columns = ['is_attributed']
    results = fit_data[['click_id']].join(y)
    results.to_pickle(results_dir+'results'+str(k)+'.pickle')
    del results
    del y
    del test_cat
    del test_feat
    del fit_data
    t2 = time.time()
    print("processes took " +str(t2-t1) +" seconds for step" +str(k+1))
    print("_________________________________________________________")
t3=time.time()
print("Whole processes took " +str(t3-t0))

loading 1th file...
file loaded. mapping categoricals
mapping done. Tokenizing now.
predicting
merging click id
processes took 88.78847098350525 seconds for step1
_________________________________________________________
loading 2th file...
file loaded. mapping categoricals
mapping done. Tokenizing now.
predicting
merging click id
processes took 88.6301281452179 seconds for step2
_________________________________________________________
loading 3th file...
file loaded. mapping categoricals
mapping done. Tokenizing now.
predicting
merging click id
processes took 88.11283755302429 seconds for step3
_________________________________________________________
loading 4th file...
file loaded. mapping categoricals
mapping done. Tokenizing now.
predicting
merging click id
processes took 87.69599938392639 seconds for step4
_________________________________________________________
loading 5th file...
file loaded. mapping categoricals
mapping done. Tokenizing now.
predicting
merging click id
proce

KeyError: "['click_id'] not in index"

#### collect results and merge

In [19]:
result_files = os.listdir('results')
frames = []
data1 = pd.DataFrame()
for l in np.arange(len(result_files)):
    if l%5 ==0:
        print(l)
    data = pd.read_pickle('.\\results\\results'+str(l)+'.pickle')
    frames =[data1,data]
    data1 = pd.concat(frames)

0
5
10
15


In [21]:
data1.to_csv('final_results.csv',index=False)